# An Example of the Horizontal Federated Analytics Task

This is an example of running horizontal federated analytics Delta Task on multiple Delta Nodes.

The data is a csv file distributed on several nodes, and the file content is the wage of  all employee in the company.
And the task is to calculate the average wage of these companies.


This example could be executed in Deltaboard directly. <span style="color:#FF8F8F;font-weight:bold">Before hitting the run button, the Delta Node API address should be modified according to the user's config, the instructions are explained in section 4 below.</span>

## 1. Import the Required Packages

We need to import Delta Task framework components from Python package ```delta-task``` including ```DeltaNode``` for Delta Node API connection, and the ```HorizontalAnalytics``` for defination of the horizontal analytics task:

In [ ]:
from delta import pandas as pd
from delta.task import HorizontalAnalytics
from delta.delta_node import DeltaNode
import delta.dataset

from typing import Dict

## 2. Define the Horizontal Federated Analytics Task

The next step is to define our horizontal analytics learning task to analyze the data on multiple nodes.

There're several parts in the PPC Task that need to be programmed by the developer:

* ***Task Config***: We can make some basis task config in the ```super().__init__()``` method. The configurations involves task name (```name```), minimum client count(```min_clients```), maximum client count(```max_clients```),waiting timeout for calculation (```wait_timeout```)，and connection timeout for each step in the procedure(```connection_timeout```).
* ***Dataset***: In the ```dataset``` method, you can specify the dataset for task. The return value is a dict of which key should be the name of dataset and value should be an instance of ```delta.dataset.DataFrame```; the key of dict should be corresponding to the parameters of the execute method. For detailed explanation of the dataset format, please refer to [this document](https://docs.deltampc.com/network-deployment/prepare-data).
* ***Analytics implementation***: We need to implement the analytics process in the execute method. The input parameters should be the same with the keys of returned dict of ```dataset``` method. Now the type of all parameter can only be ```delta.pandas.DataFrame```. ```delta.pandas.DataFrame``` is similar with ```pandas.DataFrame```,  now it support operator ```+,-,*,/,//,%```, and method```all, any, count, sum, mean, std, var, sem```.


In [ ]:
class WageAvg(HorizontalAnalytics):
    def __init__(self) -> None:
        super().__init__(
            name="wage_avg",  # The task name which is used for displaying purpose.
            min_clients=2,  # Minimum nodes required in each round, must be greater than 2.
            max_clients=3,  # Maximum nodes allowed in each round, must be greater equal than min_clients.
            wait_timeout=5,  # Timeout for calculation.
            connection_timeout=5,  # Wait timeout for each step.
        )

    def dataset(self) -> Dict[str, delta.dataset.DataFrame]:
        """
        Define the data used for analytics.
        return: a dict of which key is the dataset name and value is an instance of delta.dataset.DataFrame
        """
        return {
            "wages": delta.dataset.DataFrame("wages.csv")
        }

    def execute(self, wages: pd.DataFrame):
        """
        Implementation of analytics.
        input should be the same with keys of returned dict of dataset method.
        """
        return wages.mean()


## 3. Set the API Address of the Delta Node

After defining the task details, we're ready to run the task on the Delta Nodes.

Delta Task framework could send the task to Delta Node directly, as long as the Delta Node API address is specified.

Here we use the Delta Node API provided by Deltaboard. Deltaboard provides a separate API address for each of its users, the tasks submitted via the API could be listed inside Deltaboard. The developer could also use API from Delta Node directly.

Click "Profiles" on the sidebar of Deltaboard, copy the API Address in Deltaboard API section, and paste it here:

In [ ]:
DELTA_NODE_API = "http://127.0.0.1:6704"

## 4. Run the PPC Task

Finally we can start the task:

In [ ]:
task = WageAvg().build()
delta_node = DeltaNode(DELTA_NODE_API)
delta_node.create_task(task)

## 5. Check the Running Status

After clicking the run button, some logs will be print out showing the task is submitted to the Delta Node successfully.

To see the task execution details, go to "My Tasks" on the sidebar of Deltaboard, the task should be listed.
Click the item to view the execution logs.